In [69]:
import fitz
from tqdm import tqdm

import pandas as pd
import numpy as np

In [62]:
def text_format(text):
    cleaned_text = text.replace("\n", " ").strip()
    return cleaned_text

In [79]:
def open_read_pdf(pdf_name):
    doc = fitz.open(pdf_name)
    pages_and_text = []

    for page_num, page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_format(text)
        pages_and_text.append({"page number": page_num,
                               "page_sentence_number": len(text.split('.')),
                               "page_text": text})

    return pages_and_text 

In [83]:
pages_and_text = open_read_pdf('support_cours_bdd.pdf')
pages_and_text[46]

FileNotFoundError: no such file: 'support_cours_bdd.pdf'